In [6]:
#Importing all libraries
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [8]:
#Headers will make it look like you are using a web browser
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
#We will use the iteration to retrieve and scrape the web pages, reviews, and ratings from each page on Yelp
reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []

df = pd.read_csv('Supercuts Review List.csv')

for i, row in df.iterrows():
    if row['Brand'] == 'Sport Clips': 
        url = row['url']
        response = requests.get(url, headers=headers, verify=False).text
        soup = BeautifulSoup(response, "lxml")
    #Looping through 'div' 'review-content' will help find all the review containers we need in each page that have rating and review
        for s in soup.find_all("div", attrs={'class': 'review-content'}):
            re = s.find('p', attrs={'lang': 'en'})
    #Makes all the letters lower in reviews
            review = re.text.lower()
            review_corpus.append([review])
    #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
    #Make sure to to run the cList dict cell or else there will be an error
            expandContractions(review)
    #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
            words = word_tokenize(review)
            words = word_tokenize(review.replace('\n',' '))
            clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
            characters_to_remove = ["''",'``','...']
            clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
            english_stops = set(stopwords.words('english'))
            clean_words = [word for word in clean_words if word not in english_stops]
            wordnet_lemmatizer = WordNetLemmatizer()
            lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
            reviews.append(lemma_list)
    #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
            rating = s.find_all('img', attrs={'class': 'offscreen'})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
            rate = str(rating)
            int_rating = int(rate[11:12])

            if int_rating == 1 or int_rating == 2 or int_rating == 3:
                rating = 'neg'
                ratings.append('neg')
                neg_reviews.append(lemma_list)
            elif int_rating == 4 or int_rating == 5:
                rating = 'pos'
                ratings.append('pos')
                pos_reviews.append(lemma_list)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('haircut_review_corpus_sportclips.csv', index=False, header=False)

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/p

/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)
/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


In [9]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [10]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [11]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [12]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [17]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.001*"cut" + 0.001*"haircut" + 0.001*"great" + 0.001*"get" + 0.001*"time" '
  '+ 0.001*"hair" + 0.001*"always" + 0.001*"good" + 0.001*"place" + '
  '0.001*"sport"'),
 (1,
  '0.024*"cut" + 0.023*"great" + 0.023*"haircut" + 0.023*"hair" + 0.022*"get" '
  '+ 0.016*"time" + 0.015*"go" + 0.014*"good" + 0.013*"sport" + 0.012*"place"'),
 (2,
  '0.005*"haircut" + 0.005*"get" + 0.004*"hair" + 0.004*"cut" + 0.003*"great" '
  '+ 0.003*"sport" + 0.002*"time" + 0.002*"good" + 0.002*"always" + '
  '0.002*"come"'),
 (3,
  '0.001*"cut" + 0.001*"hair" + 0.001*"great" + 0.001*"haircut" + 0.001*"time" '
  '+ 0.001*"get" + 0.001*"go" + 0.001*"sport" + 0.001*"good" + 0.001*"place"'),
 (4,
  '0.001*"great" + 0.001*"cut" + 0.001*"haircut" + 0.001*"get" + 0.001*"hair" '
  '+ 0.001*"time" + 0.001*"place" + 0.001*"good" + 0.001*"sport" + 0.001*"go"'),
 (5,
  '0.001*"hair" + 0.001*"great" + 0.001*"haircut" + 0.001*"get" + 0.001*"clip" '
  '+ 0.001*"sport" + 0.001*"cut" + 0.001*"place" + 0.001*"go" + 0.0

In [18]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
1     -0.214164 -7.826170e-07       1        1  99.999550
5      0.047844 -1.061362e-04       2        1   0.000057
4      0.045651  1.386243e-04       3        1   0.000056
9      0.042995 -2.189176e-04       4        1   0.000055
3      0.039918  2.308327e-04       5        1   0.000054
0      0.038892 -2.080084e-04       6        1   0.000053
7      0.024384  2.708247e-04       7        1   0.000047
8      0.021969  3.522113e-05       8        1   0.000046
2     -0.012145 -2.376980e-04       9        1   0.000039
6     -0.035344  9.604004e-05      10        1   0.000038, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
1154  Default  639.000000     haircut  639.000000  30.0000  30.0000
633   Default  656.000000         cut  656.000000  29.0000  29.0000
1089  Default  617.000000         get  617.000000  28.0000  28.0000
1153  Default  623.000000        hair  623.000000  27.0000  27.0000
1131  Default  645.000000       great  645.000000  26.0000  26.0000
2684  Default  434.000000        time  434.000000  25.0000  25.0000
1104  Default  408.000000          go  408.000000  24.0000  24.0000
1110  Default  400.000000        good  400.000000  23.0000  23.0000
2476  Default  372.000000       sport  372.000000  22.0000  22.0000
1958  Default  318.000000       place  318.000000  21.0000  21.0000
2560  Default  289.000000     stylist  289.000000  20.0000  20.0000
84    Default  281.000000      always  281.000000  19.0000  19.0000
498   Default  279.000000        come  279.000000  18.0000  18.0000
473   Default  263.000000        clip  263.000000  17.0000  17.0000
1524  Default  238.000000    location  238.000000  16.0000  16.0000
2868  Default  234.000000        wait  234.000000  15.0000  15.0000
212   Default  218.000000        back  218.000000  14.0000  14.0000
1051  Default  222.000000    friendly  222.000000  13.0000  13.0000
1382  Default  201.000000         job  201.000000  12.0000  12.0000
2327  Default  204.000000     service  204.000000  11.0000  11.0000
1735  Default  186.000000         mvp  186.000000  10.0000  10.0000
915   Default  172.000000  experience  172.000000   9.0000   9.0000
2613  Default  182.000000        take  182.000000   8.0000   8.0000
1620  Default  180.000000     massage  180.000000   7.0000   7.0000
2967  Default  174.000000        year  174.000000   6.0000   6.0000
1097  Default  163.000000        give  163.000000   5.0000   5.0000
2878  Default  159.000000        want  159.000000   4.0000   4.0000
1539  Default  164.000000        look  164.000000   3.0000   3.0000
1586  Default  162.000000        make  162.000000   2.0000   2.0000
996   Default  155.000000       first  155.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
1153  Topic10    0.000057        hair  623.308716  -1.4186  -5.2102
2684  Topic10    0.000042        time  434.605865  -1.3483  -5.5005
1104  Topic10    0.000040          go  408.839081  -1.3529  -5.5662
2560  Topic10    0.000033     stylist  289.757751  -1.1924  -5.7500
1131  Topic10    0.000047       great  645.821777  -1.6414  -5.3974
1958  Topic10    0.000034       place  318.470612  -1.2555  -5.7186
2476  Topic10    0.000032       sport  372.031158  -1.4700  -5.7776
212   Topic10    0.000026        back  218.620575  -1.1670  -6.0062
1110  Topic10    0.000031        good  400.065369  -1.5697  -5.8047
915   Topic10    0.000022  experience  172.954529  -1.0774  -6.1510
498   Topic10    0.000026        come  279.068420  -1.3961  -5.9913
1524  Topic10    0.000024    location  238.500702  -1.3053  -6.0575
84    Topic10    0.000025      always  281.936279  -1.4266  -6.0115
473   Topic10    0.000025        clip  263.173584  -1.3857  -6.0395
1735  Topic10    0.000021         mvp  186.883804  -1.1944  -6.19

In [19]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.202144 -0.000003       1        1  99.999580
5     -0.079517  0.000014       2        1   0.000162
2      0.053086 -0.000311       3        1   0.000044
6      0.050447 -0.000012       4        1   0.000043
8      0.042124  0.000144       5        1   0.000036
7      0.036252  0.000269       6        1   0.000032
3      0.035819 -0.000041       7        1   0.000031
4      0.033351  0.000265       8        1   0.000030
1      0.021504 -0.000215       9        1   0.000025
0      0.009077 -0.000110      10        1   0.000024, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
1131  Default  471.000000        grind  471.000000  30.0000  30.0000
1154  Default  470.000000   haircutter  470.000000  29.0000  29.0000
633   Default  483.000000      cutting  483.000000  28.0000  28.0000
1153  Default  457.000000      haircut  457.000000  27.0000  27.0000
1089  Default  453.000000     gimmicky  453.000000  26.0000  26.0000
1110  Default  291.000000          got  291.000000  25.0000  25.0000
2684  Default  318.000000        thumb  318.000000  24.0000  24.0000
1104  Default  299.000000         gone  299.000000  23.0000  23.0000
2476  Default  271.000000       spiked  271.000000  22.0000  22.0000
1958  Default  233.000000       poster  233.000000  21.0000  21.0000
2560  Default  212.000000       studio  212.000000  20.0000  20.0000
498   Default  204.000000     commonly  204.000000  19.0000  19.0000
84    Default  206.000000       amount  206.000000  18.0000  18.0000
473   Default  193.000000        clump  193.000000  17.0000  17.0000
2868  Default  171.000000     vibrated  171.000000  16.0000  16.0000
1524  Default  175.000000     lopsided  175.000000  15.0000  15.0000
212   Default  160.000000       barley  160.000000  14.0000  14.0000
1051  Default  163.000000        fully  163.000000  13.0000  13.0000
2327  Default  149.000000        shady  149.000000  12.0000  12.0000
1382  Default  148.000000     jennifer  148.000000  11.0000  11.0000
1735  Default  136.000000          nip  136.000000  10.0000  10.0000
1620  Default  131.000000          mid  131.000000   9.0000   9.0000
2613  Default  133.000000     tackling  133.000000   8.0000   8.0000
2967  Default  127.000000        world  127.000000   7.0000   7.0000
915   Default  127.000000  explanation  127.000000   6.0000   6.0000
1539  Default  120.000000        loyal  120.000000   5.0000   5.0000
2878  Default  116.000000      visibly  116.000000   4.0000   4.0000
1586  Default  118.000000         matt  118.000000   3.0000   3.0000
2492  Default  114.000000       spread  114.000000   2.0000   2.0000
1097  Default  119.000000       glance  119.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1154  Topic10    0.000016   haircutter  470.583282  -1.9608  -5.7240
2684  Topic10    0.000013        thumb  318.914825  -1.7255  -5.8777
1131  Topic10    0.000014        grind  471.872040  -2.0589  -5.8194
1110  Topic10    0.000012          got  291.137970  -1.7522  -5.9956
1089  Topic10    0.000014     gimmicky  453.577820  -2.0710  -5.8710
84    Topic10    0.000010       amount  206.747574  -1.6193  -6.2050
1153  Topic10    0.000012      haircut  457.828522  -2.1714  -5.9620
1958  Topic10    0.000009       poster  233.523132  -1.8678  -6.3317
996   Topic10    0.000007      flowbee  113.807610  -1.3736  -6.5563
2560  Topic10    0.000008       studio  212.967667  -1.8413  -6.3973
2868  Topic10    0.000007     vibrated  171.868958  -1.6936  -6.4641
1620  Topic10    0.000007          mid  131.637726  -1.5238  -6.5609
1524  Topic10    0.000007     lopsided  175.500275  -1.7377  -6.4872
498   Topic10    0.000008     commonly  204.494522  -1.8539  -6.4505
1104  Topic10    0.000008         gone  299.435364  -2.1364  -6.35